In [1]:
import Robogame as rg
import networkx as nx
import altair as alt
import time, json
import pandas as pd
import numpy as np
import sklearn.decomposition as skd
from sklearn.linear_model import LinearRegression

To start a game server go into the server directory. Make sure you have flask and networkx installed (```conda install networkx flask flask-cors requests scipy``` should do it).

To start a server do:

```python api.py [-d directoryforgame] [-s] [-t1s T1Secret] [-t2s T2Secret] gameid```

gameid is the the prefix of all the game files (the examples we gave you are examplematch1, examplematch2, examplematch3.

```
-d/--directory is an optional directory. For example, we put the examplematch1 files in the example1 directory
-s/--simulated tells us whether to simulate team 2
-t1s/--team1secret is Team 1's secret. If you don't specify this, the server will give you one
-t2s/--team1secret is Team 2's secret. This is ignored if you use -s. If you don't specify this, the server will give you one
-m/--matchsave is a log file to save the game in, a random log name will be used if you don't specify this
```

Here's a simple game:

```python api.py -d ./example1 -s -t1s bob examplematch1```

We're using the examplematch1 files in the example1 directory, simulating player 2 and team 1's secret is 'bob' (that's us)

In [2]:
# create a game connection using bob as the "secret" key for your team (this is what you're given by the server)
game = rg.Robogame("bob")

# the default is to the localhost on port 5000, if you want something else you can do:
# game = rg.Robogame("bob",server="12.12.31.1",port=2324)

In [14]:
# tell the server we're ready to go
game.setReady()

{'Result': 'OK'}

In [22]:
game.getGameTime()

{'curtime': 5.26,
 'gameendtime_secs': 1670800026.0896583,
 'gamestarttime_secs': 1670799426.0896583,
 'servertime_secs': 1670799457.6447449,
 'unitsleft': 94.74}

In [16]:
while(True):
    gametime = game.getGameTime()
    timetogo = gametime['gamestarttime_secs'] - gametime['servertime_secs']
    
    if ('Error' in gametime):
        print("Error"+str(gametime))
        break
    if (timetogo <= 0):
        print("Let's go!")
        break
        
    print("waiting to launch... game will start in " + str(int(timetogo)))
    time.sleep(1) # sleep 1 second at a time, wait for the game to start
    

waiting to launch... game will start in 4
waiting to launch... game will start in 0
Let's go!


In [24]:
for i in np.arange(0,3):
    # this will give us just the new hints, but the object will store everything we've seen
    hints = game.getHints()
    if ('parts' in hints):
        print('we got '+str(len(hints['parts']))+' part hints')
    time.sleep(6)


# get all the hints we've collected
predHints = game.getAllPredictionHints()
partHints = game.getAllPartHints()

we got 54 part hints
we got 18 part hints
we got 12 part hints


# Visu

In [61]:
df = pd.DataFrame(game.getAllPartHints())

In [62]:
if(len(df)>0):
    df_pca = pd.DataFrame(columns=df['column'].unique(),index=df['id'].unique(),dtype=float)
    
    for i in range(len(df)):
        df_pca.loc[df.loc[i,'id'],df.loc[i,'column']] = df.loc[i,'value']

### Prodcutivity

In [63]:
df_product = pd.DataFrame(game.getRobotInfo())

In [64]:
df_pca = pd.merge(df_pca, df_product[['Productivity']].astype('float'), left_index=True, right_index=True).reset_index()

## correlation

In [143]:
for c in df_pca.columns:
    if df_pca[c].dtype==object:
        num_value = pd.DataFrame(df_pca.groupby(c).mean()['Productivity']).reset_index().rename(columns={'Productivity':'value'})
        df_pca = df_pca.merge(num_value, on=c, how='left').drop(columns=c).rename(columns={'value':c})
        
old_col = df_pca[['Productivity']]
df_pca = df_pca.drop(columns=['Productivity'])
df_pca['Productivity'] = old_col

## Multiple regression
#### problems: in the beginning, after ignore nan, there is no row leave => Error

In [121]:
#import statsmodels.api as sm
import statsmodels.formula.api as smf 
from numpy import NaN

In [127]:
col_list ='Productivity ~ '
for c in df_pca.columns[:-1]:
    col_list = col_list + '"'+c+'"'
    col_list += ' + '
col_list = col_list[:-3]

index
Astrogation Buffer Length
InfoCore Size
Sonoreceptors
Repulsorlift Motor HP
AutoTerrain Tread Count
Cranial Uplink Bandwidth
Polarity Sinks
Nanochip Model
Arakyd Vocabulator Model
Axial Piston Model


In [139]:
model = sm.OLS(df_pca['Productivity'], df_pca[df_pca.columns[:-1]], missing='drop')
results = model.fit()
results.params

ValueError: zero-size array to reduction operation maximum which has no identity

## Using linear combination of different features

In [ ]:
for c in df_pca.columns:
    if df_pca[c].dtype==object:
        num_value = pd.DataFrame(df_pca.groupby(c).mean()['Productivity']).reset_index().rename(columns={'Productivity':'value'})
        df_pca = df_pca.merge(num_value, on=c, how='left').drop(columns=c).rename(columns={'value':c})
        
old_col = df_pca[['Productivity']]
df_pca = df_pca.drop(columns=['Productivity'])
df_pca['Productivity'] = old_col

In [144]:
reg_coe = pd.DataFrame(columns=['corr'],index=df_pca.columns,dtype=float)

In [145]:
for f in df_pca.columns[:-1]:
    test = df_pca[[f,'Productivity']]
    test = test.dropna()
    if test.any()[0]:
        model = LinearRegression().fit(test[[f]], test['Productivity'])
        model.predict(df_pca[[f]].dropna())
        df_pca.loc[~df_pca[f].isna(),f+'_val'] = model.predict(df_pca[[f]].dropna())
        reg_coe.loc[f,'corr'] = model.score(test[[f]], test['Productivity'])

C:\Users\hp\anaconda3\envs\si649f22\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\hp\anaconda3\envs\si649f22\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [146]:
model = LinearRegression().fit(test[[f]], test['Productivity'])

In [147]:
df_pca['predict_all'] = 0
df_pca['predict_weight'] = 0

In [148]:
for f in reg_coe.dropna().index:
    df_pca.loc[~df_pca[f].isna(),'predict_all'] += df_pca.loc[~df_pca[f].isna(),f+'_val']*reg_coe.loc[f,'corr']
    df_pca.loc[~df_pca[f].isna(),'predict_weight'] += reg_coe.loc[f,'corr']

In [149]:
df_pca['Productivity_predict'] = df_pca['predict_all']/ df_pca['predict_weight']

In [150]:
alt.Chart(df_pca).mark_bar().encode(
    x = alt.X('index:N',sort='-y'),
    y = alt.Y("Productivity_predict:Q"),
    tooltip = alt.Tooltip(['index','Productivity_predict'])
)

C:\Users\hp\anaconda3\envs\si649f22\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [151]:
alt.Chart(reg_coe.reset_index()).mark_bar().encode(
    x = alt.X('index:N',sort='-y'),
    y = alt.Y("corr:Q"),
    tooltip = alt.Tooltip(['index','corr'])
)

C:\Users\hp\anaconda3\envs\si649f22\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

# Friendship Game

In [47]:
c1 = pd.DataFrame(game.getTree())
c1

,children,id
0,{'children': [{'children': [{'children': [{'id...,147
1,{'children': [{'children': [{'children': [{'ch...,147


In [48]:
check = False

g_id = 1
c_g_new = pd.DataFrame(columns=['id','id_'+str(g_id),'children'])
for i in range(len(c1)):
    a = c1['children'][i]
    if 'children' in a:
        children = a['children']
        check = True
    else:
        children = None
    c_g_new = c_g_new.append({'id': c1['id'][i], 'id_'+str(g_id): a['id'], 'children': children},ignore_index=True)
g_id += 1 
c1 = c_g_new

C:\Users\hp\AppData\Local\Temp\ipykernel_18128\2315041536.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id': c1['id'][i], 'id_'+str(g_id): a['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_18128\2315041536.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id': c1['id'][i], 'id_'+str(g_id): a['id'], 'children': children},ignore_index=True)


In [49]:
while(check == True):
    check = False
    c_g_new = pd.DataFrame(columns=['id_'+str(g_id-1),'id_'+str(g_id),'children'])
    for i in range(len(c1)):
        a = c1['children'][i]
        for j in range(len(a)):
            b = a[j]
            if 'children' in b:
                children = b['children']
                if len(children)>1:
                    check = True
            else:
                children = [{'id': b['id']}]
            c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
    c1 = c1.drop(columns='children').merge(c_g_new,on='id_'+str(g_id-1),how='outer')  
    g_id += 1
c1 = c1.drop(columns='children')

C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignor

C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignor

C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignor

C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignor

C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignor

C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignor

C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_18128\1358336334.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c_g_new = c_g_new.append({'id_'+str(g_id-1): c1['id_'+str(g_id-1)][i], 'id_'+str(g_id): b['id'], 'children': children},ignor

In [50]:
predict = pd.DataFrame(game.getAllPredictionHints())
predict = predict.rename(columns={'id':'id_data'})

In [51]:
c1['id'] = c1['id_9']

In [52]:
Friendship_df = pd.DataFrame(columns=['id_data','G_diff','time','value','id'])

In [53]:
for i in range(1,len(c1.columns)):
    Friendship = pd.merge(predict,c1,left_on='id_data',right_on='id_'+str(i),how='inner')
    Friendship['G_diff'] = len(c1.columns)-i-1
    Friendship_df = pd.concat([Friendship_df,Friendship[['id_data','G_diff','time','value','id']]])

In [54]:
id_check = 8
Friendship_df_app = Friendship_df[Friendship_df['id']==id_check]
alt.Chart(Friendship_df_app).mark_circle().encode(
    x = alt.X('time:Q',sort='-y'),
    y = alt.Y("value:Q"),
    color = alt.Color('G_diff:O'),
    tooltip = alt.Tooltip(['time','value'])
)

C:\Users\hp\anaconda3\envs\si649f22\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)